In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from math import ceil
import os

In [2]:
nTurn = 1024

# 1. Preparation

In [3]:
beam, lattice = impact.readInputFile('test_IOTA.in')

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [4]:
ke   = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

### adjust lattice

In [5]:
# L=0.0
# for i in range(len(lattice)):
#     if 'length' in lattice[i].keys():
#         L = L+lattice[i].length
# print(L)

# for i in range(len(lattice)):
#     if 'length' in lattice[i].keys():
#         lattice[i].n_sckick = ceil(lattice[i].length*20)
#         if lattice[i].type in ['nonlinear_insert','dipole']:
#             lattice[i].n_map = 10
#         else:
#             lattice[i].n_map = 2
        
lattice = [ item for item in lattice if not item.type == 'write_raw_ptcl' ]
lattice = [ item for item in lattice if not item.type == '-8' ]

TBT = impact.getElem('TBT')
TBT.file_id = nTurn+100000
lattice.insert(1,TBT)

# lattice.append(impact.getElem('write_raw_ptcl'))
# lattice[-1].turn = nTurn
# lattice[-1].file_id = -nTurn-100000

L=0.0
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        L = L+lattice[i].length
print(L)

39.968229715800106


### backward lattice

In [6]:
latticeB = copy(lattice[::-1])
latticeB.insert(0,copy(lattice[0]))
latticeB.pop()
latticeB[0].turns = nTurn

for i in range(len(latticeB)):
    if 'length' in latticeB[i]:
        latticeB[i].length = -latticeB[i].length
    if latticeB[i].type == 'dipole':
        latticeB[i].bending_angle            =-latticeB[i].bending_angle
        latticeB[i].fringe_field_integration =-latticeB[i].fringe_field_integration
    if 'KL_sext' in latticeB[i]:
#         print(latticeB[i])
        latticeB[i].KL_sext = -latticeB[i].KL_sext

In [7]:
i = 1
print(latticeB[-i])
print(lattice[i])


          type: 'TBT'
       file_id: 101024
          turn: 1
 sample_period: 1

          type: 'TBT'
       file_id: 101024
          turn: 1
 sample_period: 1


### write Impact input file

In [8]:
beam.distribution.distribution_type = 'ReadFile_binary'
beam.distribution.file_id = nTurn
beam.nCore_y = 48
beam.nCore_z = 32
impact.writeInputFile(beam,latticeB,'test.in.'+str(nTurn)+'.backward')